In [ ]:
import sys, os
if 'google.colab' in sys.modules:
      if os.getcwd() == '/content':
        !git clone 'https://github.com/pgruening/em_football.git'
        !pip install git+https://github.com/pgruening/dlbio
        !pip install mdutils
        os.chdir('em_football')

In [ ]:
from os.path import join

import matplotlib.pyplot as plt
import numpy as np
import torch
from dataset.ds_qualification import get_one_hot
from DLBio.helpers import load_json
from DLBio.pytorch_helpers import get_device
from game_plan_api import get_winner, run
from helpers import load_model
from tqdm import tqdm

In [ ]:
PATH = 'used_model'


class MyModel():
    def __init__(self, max_lk):
        device = 'cpu'
        options = load_json(join(PATH, 'opt.json'))
        device = get_device()
        model = load_model(
            options, device, new_model_path=join(PATH, 'model.pt'),
            map_location=torch.device(device)
        )
        to_one_hot = get_one_hot()
        model.to_one_hot = to_one_hot
        model.d = device
        self.model = model
        self.max_lk = max_lk

    def __call__(self, team_1, team_2, no_draws=False):
        with torch.no_grad():
            out = self.model.predict(
                team_1, team_2, no_draws=no_draws, max_lk=self.max_lk)
        return out


In [ ]:
def main(path='my_results.md'):
    model = MyModel(True)
    group_tables, all_games = run(model, path=path)
    return group_tables, all_games

def winner_dist(path='my_winner_dist.png'):
    model = MyModel(False)
    winners_ = []
    for _ in tqdm(range(100)):
        winner = get_winner(model)
        winners_.append(winner)

    plt.hist(np.array(winners_))
    plt.show()


In [ ]:
def main(path='my_results.md'):
    model = MyModel(True)
    group_tables, all_games = run(model, path=path)
    return group_tables, all_games


In [ ]:
group_tables, all_games = main()

In [ ]:
for gt in group_tables.values():
    display(gt)

In [ ]:
all_games

In [ ]:
winner_dist()